<a href="https://colab.research.google.com/github/MDankloff/Cluster-Bias-Disco/blob/main/BAF_Reproduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
!pip install google-colab

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
!pip install aequitas

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 7.8 MB/s eta 0:00:00


In [2]:
!pip install pyarrow

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgbm
import yaml              # Read hyperparameter space configuration
import glob
import os

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder # Categorical encoding for LGBM models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

from aequitas.group import Group                # Fairness metrics
from random_search import RandomValueTrial, suggest_callable_hyperparams  # Random search wrapper methods

In [17]:
import yaml

# If the file is within a subdirectory, for example, named "config":
with open("/content/lightgbm_hyperparameter_space.yaml", "r") as file:
    # Remove the 'Loader' argument
    hyperparams = yaml.safe_load(file)

# Load Bank Account Fraud (BAF) Tabular Dataset Suite

In [19]:
base_path = '/content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF'

# Dictionary mapping dataset names to CSV paths
datasets_paths = {
    "Base": base_path + "/Base.csv",
    "Variant I": base_path + "/Variant I.csv",
    "Variant II": base_path + "/Variant II.csv",
    "Variant III": base_path + "/Variant III.csv",
    "Variant IV": base_path + "/Variant IV.csv",
    "Variant V": base_path + "/Variant V.csv",
}

# Convert CSV files to Parquet
for variant, csv_path in datasets_paths.items():
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)

    # Create the corresponding Parquet path by replacing .csv with .parquet
    parquet_path = csv_path.replace(".csv", ".parquet")

    # Save the DataFrame as a Parquet file
    df.to_parquet(parquet_path, engine='pyarrow')

    print(f"Converted {variant} to Parquet at {parquet_path}")


Converted Base to Parquet at /content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF/Base.parquet
Converted Variant I to Parquet at /content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF/Variant I.parquet
Converted Variant II to Parquet at /content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF/Variant II.parquet
Converted Variant III to Parquet at /content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF/Variant III.parquet
Converted Variant IV to Parquet at /content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF/Variant IV.parquet
Converted Variant V to Parquet at /content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF/Variant V.parquet


In [21]:
base_path = '/content/drive/MyDrive/Mirthe Supervision Map /Paper 3a: FC Bias Disco/BAF'

extension = "parquet" #or smaller "parquet" depending on the downloaded file
data_paths = glob.glob(f"{base_path}/*.{extension}")

def read_dataset(path, ext = extension):
    if ext == "csv":
      return pd.read_csv(path)
    elif ext == "parquet":
      return pd.read_parquet(path)
    else:
      raise ValueError(f"Unsupported file extension: {ext}")

# Extract variant name from the file path (without the extension)
def get_variant(path):
    return os.path.basename(path).split(".")[0]

# Dictionary comprehension to read all CSV files into a dictionary of DataFrames
dataframes = {
    get_variant(path): read_dataset(path) for path in data_paths
}
print(f"Loaded datasets: {list(dataframes.keys())}")

datasets_paths = {
    "Base": base_path + "/Base.csv", # sampled to best represent original dataset
    "Variant I": base_path + "/Variant I.parquet", # higher group size disparity than base - reducing the size of the minority group from approx 20 - 10% of the dataset
    "Variant II": base_path + "/Variant II.parquet", # higher prevalance disparity than base - one group has 5 x the fraud detection rate of the other while group sizes are equal
    "Variant III": base_path + "/Variant III.parquet", # better separability for one of the groups -
    "Variant IV": base_path + "/Variant IV.parquet", # higher prevalance disparity in train
    "Variant V": base_path + "/Variant V.parquet", # better separability in train for one of the groups
}

Loaded datasets: ['Base', 'Variant I', 'Variant II', 'Variant III', 'Variant IV', 'Variant V']


In [28]:
datasets = {key: read_dataset(path) for key, path in datasets_paths.items()}

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [25]:
dataframes["Variant I"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   prev_address_months_count         1000000 non-null  int64  
 4   current_address_months_count      1000000 non-null  int64  
 5   customer_age                      1000000 non-null  int64  
 6   days_since_request                1000000 non-null  float64
 7   intended_balcon_amount            1000000 non-null  float64
 8   payment_type                      1000000 non-null  object 
 9   zip_count_4w                      1000000 non-null  int64  
 10  velocity_6h                       1000000 non-null  float64
 11  velocity_24h                      1000

5 LightGBM models with the best results (evaluation metrics accuracy or AUC) were chosen. These were trained and evaluated on the original data and involved testing 200 different sets hps (e.g., learning rate, number of leaves, max depth) that control how the model learns.
The HP configurations were chosen through random search.